In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/home/andrey/ds_bootcamp/Phase_0/Week_3/P0_W3_D4_property_project/_data.csv', sep=',', index_col=0)

#Оставляем только Город Москва, все остальные строчки удаляем
df = df[df['Адрес'].str.contains('Москва', case=False, na=False)]

# Извлечение первого числа (этаж)
df['Этаж'] = df['Дом'].apply(lambda x: x.split('/')[0] if pd.notna(x) else None)

# Извлечение второго числа (этажность)
df['Этажность'] = df['Дом'].apply(lambda x: x.split('/')[1].split(',')[0] if pd.notna(x) and '/' in x else None)

# Оставляем только первую часть строки до первого пробела (цифры)
df['Количество комнат'] = df['Количество комнат'].apply(lambda x: x.split(",")[0] if pd.notna(x) else x)

# Преобразуем колонку в числовой тип
df['Количество комнат'] = df['Количество комнат'].astype(float)

# Преобразуем колонку в число
df['Цена'] = df['Цена'].apply(lambda x: '{:,.0f}'.format(float(x.split(' ')[0])))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Ремонт, Саузел, дети/животные, Доплонительно, Серия Дома == NAN == 0
# Кол. комнат==диапозон площади =>комнат, Площать комнат==2-ое Жилая площать/каол. комнат если есть то Общая площ/кол. комнат, Высота потлков==Мидиана по ЖК

# Парковка==ЖК-Роман, 
# Лифт==ЖК-Роман, 
# Мусоропровод==ЖК-Роман,

# Метро==Адрес/Нзавние ЖК-Андрей,
# Нзавние ЖК==Адрес/Метро-Андрей, 
 
# Балкон==Описание-Андрей, 
# Окна==Описание-Андрей

ID  объявления                  0
Количество комнат             535
Тип                             0
Метро                         346
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    11174
Цена                            0
Телефоны                        0
Описание                        0
Ремонт                       2463
Площадь комнат, м2           7228
Балкон                       6630
Окна                         5150
Санузел                      2041
Можно с детьми/животными     4915
Дополнительно                 272
Название ЖК                 15281
Серия дома                  17646
Высота потолков, м           9202
Лифт                         4192
Мусоропровод                 8007
Ссылка на объявление            0
Этаж                            0
Этажность                       0
dtype: int64

In [4]:
#Санузел
#  Заполняем колонку "Санузел" на основе ключевых слов в колонке "Описание"
df['Санузел'] = df.apply(lambda row: 'Раздельный' if 'раздельный санузел' in str(row['Описание']).lower()
                         else 'Совмещенный' if 'совмещенный санузел' in str(row['Описание']).lower()
                         else 'Два санузла' if 'два санузла' in str(row['Описание']).lower()
                         else 0, axis=1)

In [5]:
#Можно с детьми/животными
#  Заполняем колонку "Можно с детьми/животными" на основе ключевых слов в колонке "Описание"
df['Можно с детьми/животными'] = df.apply(lambda row: 'Можно с детьми' if 'можно с детьми' in str(row['Описание']).lower()
                                          else 'Можно с животными' if 'можно с животными' in str(row['Описание']).lower()
                                          else 'Нельзя с животными' if 'без животных' in str(row['Описание']).lower()
                                          else 0, axis=1)

In [6]:
# Ремонт
# Заполняем колонку "Ремонт" на основе ключевых слов в колонке "Описание"
df['Ремонт'] = df.apply(lambda row: 'Евроремонт' if 'евроремонт' in str(row['Описание']).lower()
                        else 'Косметический ремонт' if 'косметический ремонт' in str(row['Описание']).lower()
                        else 'Без ремонта' if 'без ремонта' in str(row['Описание']).lower()
                        else 0, axis=1)


In [7]:
#Балкон
df['Балкон'] = df.apply(lambda row: row['Описание'] 
                        if 'балкон' in str(row['Описание']).lower() or 'лоджия' in str(row['Описание']).lower() 
                        else 0, axis=1)

In [8]:
#Окна
df['Окна'] = df.apply(lambda row: row['Описание'] 
                        if 'на улицу' in str(row['Описание']).lower() 
                        or 'во двор' in str(row['Описание']).lower() 
                        or 'на улицу и двор' in str(row['Описание']).lower()
                        else 0, axis=1)

In [9]:
#Метро
#1. Создаем новую колонку, в которой оставляем только части [1] и [2] после разбивки в строке чтоб оставить только улицу и номер дома
df['Краткий Адрес'] = df['Адрес'].apply(lambda x: ', '.join(x.split(',')[1:3]).strip() if len(x.split(',')) > 2 else x)

#2. Создаем словарь, где для каждого "Краткий Адрес" указывается метро (где метро не пустое)
address_metro_mapping = df.dropna(subset=['Метро']).set_index('Краткий Адрес')['Метро'].to_dict()

#3. Заполняем пропуски в колонке "Метро" на основе словаря соответствий
df['Метро'] = df.apply(lambda row: address_metro_mapping.get(row['Краткий Адрес'], row['Метро']), axis=1)

#4. Создаем словарь соответствий ЖК -> Метро (где метро не пустое)
zhk_metro_mapping = df.dropna(subset=['Метро']).set_index('Название ЖК')['Метро'].to_dict()

#5. Заполняем пропуски в колонке "Метро" на основе названия ЖК
df['Метро'] = df.apply(lambda row: zhk_metro_mapping.get(row['Название ЖК'], row['Метро']), axis=1)

# 6. Заполняем оставшиеся пробелы в колонке "Метро" значением 0
df['Метро'] = df['Метро'].fillna(0)

In [10]:
#Название ЖК
# 1. Создаем словарь соответствий Краткий Адрес -> Название ЖК (где название ЖК не пустое)
address_to_zhk_mapping = df.dropna(subset=['Название ЖК']).set_index('Краткий Адрес')['Название ЖК'].to_dict()

# 2. Заполняем пропуски в колонке "Название ЖК" на основе "Краткий Адрес"
df['Название ЖК'] = df.apply(lambda row: address_to_zhk_mapping.get(row['Краткий Адрес'], row['Название ЖК']), axis=1)

df['Название ЖК'] = df['Название ЖК'].fillna(0)

In [11]:
#Лифт
# Заполняем пропуски в колонке "Лифт" значением 1, если в колонке "Название ЖК" не NaN
df['Лифт'] = df.apply(lambda row: "Есть" if pd.notna(row['Название ЖК']) and pd.isna(row['Лифт']) else row['Лифт'], axis=1)

# Создаем словарь соответствий Краткий Адрес -> Лифт (где Лифт не пустое)
address_lift_mapping = df.dropna(subset=['Лифт']).set_index('Краткий Адрес')['Лифт'].to_dict()

# Заполняем пропуски в колонке "Лифт" на основе "Краткий Адрес"
df['Лифт'] = df.apply(lambda row: address_lift_mapping.get(row['Краткий Адрес'], row['Лифт']), axis=1)

# Заполняем колонку "Лифт" значением "Есть", если этажность больше или равна 6
df['Лифт'] = df.apply(lambda row: 'Есть' if pd.notna(row['Этажность']) and int(row['Этажность']) >= 6 else row['Лифт'], axis=1)

# Заполняем колонку "Лифт" значением "Есть", если в колонке "Описание" есть слово "лифт"
df['Лифт'] = df.apply(lambda row: 'Есть' if 'лифт' in str(row['Описание']).lower() else row['Лифт'], axis=1)

df['Лифт'] = df['Лифт'].fillna(0)


In [12]:
#Мусоропровод
# 1. Заполняем пропуски в колонке "Мусоропровод" значением "Есть", если в колонке "Название ЖК" не NaN
df['Мусоропровод'] = df.apply(lambda row: "Есть" if pd.notna(row['Название ЖК']) and pd.isna(row['Мусоропровод']) else row['Мусоропровод'], axis=1)

# 2. Создаем словарь соответствий Краткий Адрес -> Мусоропровод (где Мусоропровод не пустое)
address_trash_mapping = df.dropna(subset=['Мусоропровод']).set_index('Краткий Адрес')['Мусоропровод'].to_dict()

# Заполняем пропуски в колонке "Мусоропровод" на основе "Краткий Адрес"
df['Мусоропровод'] = df.apply(lambda row: address_trash_mapping.get(row['Краткий Адрес'], row['Мусоропровод']), axis=1)

# 3. Заполняем колонку "Мусоропровод" значением "Есть", если в колонке "Описание" есть слово "мусоропровод"
df['Мусоропровод'] = df.apply(lambda row: 'Есть' if 'мусоропровод' in str(row['Описание']).lower() else row['Мусоропровод'], axis=1)

df['Мусоропровод'] = df['Мусоропровод'].fillna(0)


In [13]:
#Парковка
# 1. Заполняем пропуски в колонке "Парковка" значением "Есть", если в колонке "Название ЖК" не NaN
df['Парковка'] = df.apply(lambda row: "Есть" if pd.notna(row['Название ЖК']) and pd.isna(row['Парковка']) else row['Парковка'], axis=1)

# 2. Создаем словарь соответствий Краткий Адрес -> Парковка (где Парковка не пустое)
address_parking_mapping = df.dropna(subset=['Парковка']).set_index('Краткий Адрес')['Парковка'].to_dict()

# Заполняем пропуски в колонке "Парковка" на основе "Краткий Адрес"
df['Парковка'] = df.apply(lambda row: address_parking_mapping.get(row['Краткий Адрес'], row['Парковка']), axis=1)

# 3. Заполняем колонку "Парковка" значением "Есть", если в колонке "Описание" есть слово "парковка"
df['Парковка'] = df.apply(lambda row: 'Есть' if 'парковка' in str(row['Описание']).lower() else row['Парковка'], axis=1)

df['Парковка'] = df['Парковка'].fillna(0)

In [14]:
#Количество комнат
# Разделяем колонку "Площадь, м2" по символу "/"
df[['Общая площадь', 'Жилая площадь', 'Площадь кухни']] = df['Площадь, м2'].str.split('/', expand=True)

# Преобразуем новые колонки в числовой тип (если это нужно)
df[['Общая площадь', 'Жилая площадь', 'Площадь кухни']] = df[['Общая площадь', 'Жилая площадь', 'Площадь кухни']].astype(float)

# Заполняем пропуски в колонке "Жилая площадь" значением, равным половине "Общей площади"
df['Жилая площадь'] = df['Жилая площадь'].fillna(df['Общая площадь'] / 2)

# Шаг 1: Находим медианную (или среднюю) Жилую площадь для каждой группы по количеству комнат
median_living_area = df.groupby('Количество комнат')['Жилая площадь'].median()

# Шаг 2: Функция для заполнения пропусков на основе Жилой площади
def fill_room_count(row, median_living_area):
    if pd.isna(row['Количество комнат']):
        for rooms, median_area in median_living_area.items():
            if row['Жилая площадь'] <= median_area:
                return rooms
        return median_living_area.idxmax()  # Если площадь больше всех медианных значений, возвращаем максимальное количество комнат
    return row['Количество комнат']

# Применяем функцию для заполнения пропусков
df['Количество комнат'] = df.apply(lambda row: fill_room_count(row, median_living_area), axis=1)

df['Количество комнат'] = df['Количество комнат'].fillna(0)

In [15]:
# #Площадь комнат, м2
# Проверяем, что в колонках "Жилая площадь" и "Количество комнат" нет пропусков
df['Площадь комнат, м2'] = df['Площадь комнат, м2'].fillna(df['Жилая площадь'] / df['Количество комнат'])

df['Площадь комнат, м2'] = df['Площадь комнат, м2'].fillna(0)

In [16]:
#Высота потолков
# 1: Исключаем строки, где "Высота потолков, м" NaN, чтобы избежать предупреждений
df_non_nan = df.dropna(subset=['Высота потолков, м'])

# 2: Найти медианные значения высоты потолков для каждого ЖК (без NaN значений)
median_ceiling_by_jk = df_non_nan.groupby('Название ЖК')['Высота потолков, м'].transform('median')

# 3: Заполняем пропуски в колонке "Высота потолков, м" медианными значениями для каждого ЖК
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(median_ceiling_by_jk)

# 1: Создаем словарь соответствий 'Краткий Адрес' -> 'Высота потолков, м' (где значение не NaN)
address_ceiling_mapping = df.dropna(subset=['Высота потолков, м']).set_index('Краткий Адрес')['Высота потолков, м'].to_dict()

# 2: Функция для заполнения NaN значений в колонке 'Высота потолков, м' на основе 'Краткий Адрес'
df['Высота потолков, м'] = df.apply(lambda row: address_ceiling_mapping.get(row['Краткий Адрес'], row['Высота потолков, м']) if pd.isna(row['Высота потолков, м']) else row['Высота потолков, м'], axis=1)

# 1: Вычисляем медиану для колонки "Высота потолков, м"
median_ceiling = df['Высота потолков, м'].median()

# 2: Заполняем оставшиеся NaN медианным значением
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(median_ceiling)

In [17]:
#Площадь кухни
#  Шаг 1: Вычисляем медиану для колонки "Площадь кухни"
median_kitchen_area = df['Площадь кухни'].median()

# Шаг 2: Заполняем пропуски в колонке "Площадь кухни" медианным значением
df['Площадь кухни'] = df['Площадь кухни'].fillna(median_kitchen_area)

In [19]:
#Серия дома и Дополнительно
df['Дополнительно'] = df['Дополнительно'].fillna(0)
df['Серия дома'] = df['Серия дома'].fillna(0)

In [23]:
#Переименование колонок
columntranslation = {
    'ID  объявления': 'ID',    
    'Количество комнат': 'Number_of_Rooms',
    'Тип': 'Type',
    'Метро': 'Metro',
    'Адрес': 'Address',
    'Площадь, м2': 'Area_m2',
    'Дом': 'House',
    'Парковка': 'Parking',
    'Цена': 'Price',
    'Телефоны': 'Phones',
    'Описание': 'Description',
    'Ремонт': 'Renovation',
    'Площадь комнат, м2': 'Room_Area_m2',
    'Балкон': 'Balcony',
    'Окна': 'Windows',
    'Санузел': 'Bathroom',
    'Можно с детьми/животными': 'Pets/Children_Allowed',
    'Дополнительно': 'Additional_Info',
    'Название ЖК': 'HousingСomplex',
    'Серия дома': 'House_Series',
    'Высота потолков, м': 'Ceiling_Height_m',
    'Лифт': 'Elevator',
    'Мусоропровод': 'Garbage_Chute',
    'Ссылка на объявление': 'Ad_Link',
    'Этаж': 'Floor',
    'Этажность': 'Total_Floors',
    'Краткий Адрес': 'Short_Address',
    'Цена числовая': 'Price_Numeric',
    'Количество комнат числовое': 'Rooms_Numeric',
    'Жилая площадь, м2': 'Living_Area_m2',
    'Общая площадь': 'Total_area',
    'Жилая площадь': 'Living_area',
    'Площадь кухни': 'Kitchen_area'
}

df.rename(columns=columntranslation, inplace=True)

# Рабочие колонки Выше

In [25]:
#Сохранение фаил data.csv
df.to_csv('data.csv', index=False)